<a href="https://www.kaggle.com/code/dascient/cryptoforecast-univariate-lstm-rnn-approach?scriptVersionId=216315614" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Authors: dtad & jcre
## Inspiration: https://machinelearningmastery.com/time-series-forecasting-long-short-term-memory-network-python/

# Welcome to our Universe
## [@donutz.ai](www.donutz,ai/#)


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
from IPython.display import clear_output
clear_output()
import time,csv
import matplotlib.pyplot as plt
from dateutil.tz import tzlocal
from datetime import datetime
import seaborn as sns
from PIL import Image
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# load kaggle data

In [ ]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        clear_output()

# could've done this better, but hey...
# for kaggle
# rootdir = '/kaggle/input/g-research-crypto-forecasting/'

# for home
rootdir = '/kaggle/input/g-research-crypto-forecasting/'

# data pull
example_sample_submission = pd.read_csv(rootdir+'example_sample_submission.csv')
asset_details = pd.read_csv(rootdir+'asset_details.csv')
example_test = pd.read_csv(rootdir+'example_test.csv')
train = pd.read_csv(rootdir+'train.csv')
supplemental_train = pd.read_csv(rootdir+'supplemental_train.csv')

print('csv/files loaded')
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# timestamping 
supplemental_train['timestamp'] = supplemental_train.timestamp.astype('datetime64[s]')
example_test['timestamp'] = example_test.timestamp.astype('datetime64[s]')
train['timestamp'] = train.timestamp.astype('datetime64[s]')
print('timestamping complete')

# begin tensorflow application

[](http://)

In [ ]:
import numpy
import matplotlib.pyplot as plt
import math
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [ ]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return numpy.array(dataX), numpy.array(dataY)

# fix random seed for reproducibility
numpy.random.seed(7)

In [ ]:
# load the dataset
# isolate to only one asset, for now...
# this also assumes only one variable - & we do not expect to output a complete "submission.csv"
dataframe = supplemental_train[supplemental_train.Asset_ID == 8][['Close']].reset_index(drop=True).head(100)

dataset = dataframe.values
dataset = dataset.astype('float32')

In [ ]:
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

In [ ]:
# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

# reshape into X=t and Y=t+1
look_back = 3
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

# reshape input to be [samples, time steps, features]
trainX = numpy.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
testX = numpy.reshape(testX, (testX.shape[0], testX.shape[1], 1))

In [ ]:
# create and fit the LSTM network
# could change batch_size
batch_size = 1

model = Sequential()
model.add(LSTM(4, batch_input_shape=(batch_size, look_back, 1), stateful=True, return_sequences=True))
model.add(LSTM(4, batch_input_shape=(batch_size, look_back, 1), stateful=True))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
clear_output()

In [ ]:
# trains _ samples 10x 
for i in range(10):
    model.fit(trainX, trainY, epochs=1, batch_size=batch_size, verbose=2, shuffle=False)
    model.reset_states()

clear_output()

# make predictions
trainPredict = model.predict(trainX, batch_size=batch_size)
model.reset_states()
testPredict = model.predict(testX, batch_size=batch_size)

# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

In [ ]:
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.6f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.6f RMSE' % (testScore))
# shift train predictions for plotting
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
# shift test predictions for plotting
testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict
# plot baseline and predictions
plt.figure(figsize=(18,7))
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

# Submission

In [ ]:
#import gresearch_crypto
#env = gresearch_crypto.make_env()

# Training data is in the competition dataset as usual
#train_df = pd.read_csv('/kaggle/input/g-research-crypto-forecasting/train.csv', low_memory=False)

# load model
#model.fit(train_df)
#model.fit(train_df)

#iter_test = env.iter_test()

#for (test_df, sample_prediction_df) in iter_test:
#    sample_prediction_df['Target'] = model.predict(test_df)
#    env.predict(sample_prediction_df)


# submission
#sample_prediction_df['Target'].to_csv('submission.csv', index=False)
print("project complete, for now.")

In [ ]:
# still need output: submission.csv

# to be continued...